# Interpolation convertor

**So Many Interpolations—What to Choose?**

In the `samplers_2d_toys`, we mentioned that the interpolation coefficients $\alpha_t, \beta_t$ can be any time-differentiable functions satisfying $\alpha_0 = \beta_1 = 0$ and $\alpha_1 = \beta_0 = 1$. Typically, we also require $\alpha_t$ to be monotonically increasing and $\beta_t$ to be monotonically decreasing. We call this scheme *affine interpolations*. The interpolation process for *affine interpolations* is then defined as:

$$
\begin{align}
	X_t &= \alpha_t \cdot X_0 + \beta_t \cdot X_1 \\
	\dot X_t &= \dot \alpha_t \cdot X_0 + \dot \beta_t \cdot X_1 \\
\end{align}
$$

The rectified flow velocity is given by:

$$
v(z, t) = \mathbb{E}[ \dot{X}_t \mid X_t = z] = \arg \min_v \int_0^1 \mathbb{E} \left[\left\| \dot{\alpha}_t X_1 + \dot{\beta}_t X_0 - v(X_t, t) \right\|^2 \right] \, \mathrm{d}t
$$

At first glance, the velocity appears to be intrinsically connected with the interpolation coefficients, suggesting that the interpolation scheme must be chosen during training. However, in this section, we will demonstrate that it is actually possible to convert between different affine interpolation schemes at inference time, without the need to retrain the model.

Let's first train a rectified flow with straight interpolation.

In [ ]:
import torch
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import warnings
import copy
import plotly.graph_objects as go

import torch.distributions as dist

from rectified_flow.utils import set_seed
from rectified_flow.datasets.toy_gmm import TwoPointGMM

from rectified_flow.rectified_flow import RectifiedFlow
from rectified_flow.models.toy_mlp import MLPVelocityConditioned, MLPVelocity

set_seed(0)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
from rectified_flow.datasets.toy_gmm import TwoPointGMM

n_samples = 50000
pi_0 = dist.MultivariateNormal(torch.zeros(2, device=device), torch.eye(2, device=device))
pi_1 = TwoPointGMM(x=15.0, y=2, std=0.3)
D0 = pi_0.sample([n_samples])
D1, labels = pi_1.sample_with_labels([n_samples])
labels.tolist()

from rectified_flow.flow_components.interpolation_solver import AffineInterp
from rectified_flow.utils import visualize_2d_trajectories_plotly

straight_interp = AffineInterp("straight")
spherical_interp = AffineInterp("spherical")

t = torch.linspace(0, 1, 1000)

idx = torch.randperm(n_samples)[:1000]
x_0 = D0[idx]
x_1 = D1[idx]

print(x_0.shape)

straight_interp_list = []
spherical_interp_list = []

for t in np.linspace(0, 1, 50):
	x_t_straight, dot_x_t_straight = straight_interp.forward(x_0, x_1, t)
	x_t_spherical, dot_x_t_spherical = spherical_interp.forward(x_0, x_1, t)
	straight_interp_list.append(x_t_straight)
	spherical_interp_list.append(x_t_spherical)

visualize_2d_trajectories_plotly(
	trajectories_dict={"straight interp": straight_interp_list, "spherical interp": spherical_interp_list},
	D1_gt_samples=D1[:5000],
	num_trajectories=100,
	title="Interpolated Trajectories Visualization",
)

In [ ]:
def rf_trainer(rectified_flow, label = "loss", batch_size = 1024):
    model = rectified_flow.velocity_field
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

    losses = []
    for step in range(5000):
        optimizer.zero_grad()
        idx = torch.randperm(n_samples)[:batch_size]
        x_0 = D0[idx].to(device)
        x_1 = D1[idx].to(device)

        loss = rectified_flow.get_loss(x_0, x_1)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())

        if step % 200 == 0:
            print(f"Epoch {step}, Loss: {loss.item()}")

    plt.plot(losses, label=label)
    plt.legend()

In [ ]:
from rectified_flow.models.toy_mlp import MLPVelocity

straight_rf = RectifiedFlow(
    data_shape=(2,),
    velocity_field=MLPVelocity(2, hidden_sizes = [128, 128, 128]).to(device),
    interp=straight_interp,
    source_distribution=pi_0,
    device=device,
)

rf_trainer(straight_rf, "straight interp")

In [ ]:
from rectified_flow.samplers import EulerSampler

euler_sampler_straight = EulerSampler(straight_rf, num_steps=100)

euler_sampler_straight.sample_loop(seed=0, num_samples=1000)

visualize_2d_trajectories_plotly(
	trajectories_dict={"1rf straight": euler_sampler_straight.trajectories},
	D1_gt_samples=D1[:5000],
	num_trajectories=100,
	title="Euler Sampler Visualization",
)

Consider two affine interpolation processes defined with same coupling $(X_0, X_1)$:
$$
X_t = \alpha_t X_1 + \beta_t X_0 \quad \text{and} \quad X_{t'}' = \alpha_{t'}' X_1 + \beta_{t'}' X_0.
$$
Our goal is to convert the rectified flow induced by $\{X_t\}$ into the one induced by $\{X_{t'}'\}$. The conversion procedure involves first matching the ratio $\alpha/\beta$ and then adjusting the scale accordingly.

1. Matching Time $t$:

   Note that
   $$
   \dot{\alpha}_t > 0, \quad \dot{\beta}_t < 0, \quad \alpha_t, \beta_t \in [0, 1], \quad \forall t \in [0, 1]
   $$
   This implies that $\alpha_t / \beta_t$ is strictly increasing on $[0,1]$. Hence, for any given $t$, there exists a unique $t'$ such that:
   $$
   \frac{\alpha_t}{\beta_t} = \frac{\alpha'_{t'}}{\beta'_{t'}}.
   $$
   Similarly, for any given $t'$, there exists a unique $t$. That said, $t$ and $t'$ can be uniquely matched in both directions. 

2. Match Scales:

   After matching the ratios, consider:
   $$
   \frac{X_t}{X_{t'}'} = \frac{\alpha_tX_1+\beta_tX_0}{\alpha'_{t'}X_1+\beta'_{t'}X_0}= \frac{\alpha_t}{\alpha'_{t'}} \cdot \frac{X_1+\beta_t / \alpha_t X_0}{X_1 + \beta'_{t'}/\alpha'_{t'}X_0}=\frac{\alpha_t}{\alpha'_{t'}}.
   $$
   Define the scaling factor:
   $$
   \omega_t := \frac{\alpha_t}{\alpha'_{t'}} = \frac{\beta_t}{\beta'_{t'}}
   $$

Given a pretrained 1-recitified flow induced from $\{X_t\}$, we want to transform it into the recitied flow induced from $\{X_t'\}$, and then solve the corresponding ODE for the transformed trajectories $\{Z_{t'}'\}$. Because the same transformation applies between $\{Z_t\}$ and $\{Z_{t'}'\}$ as between $\{X_t\}$ and $\{X_{t'}'\}$, we can proceed as follows:

1. Use a binary search to find $t$ that matches the $\alpha/\beta$ ratio for each given $t$'. (As we're solving the ODE in the transformed space $\{Z_{t'}'\}$)
2. Compute the scaling factor $\omega_t$.
3. Transform the velocity from the original rectified flow $\{Z_t\}$ to the new coordinates $\{Z_{t'}'\}$.

In [ ]:
class AffineInterpConverter:
    def __init__(self, rectified_flow, target_interp):
        self.rectified_flow = rectified_flow
        self.source_interp = rectified_flow.interp
        self.target_interp = target_interp

    def match_time_and_scale(self, target_t):
        """Given t' from target_interp, return corresponding t from source_interp and scaling factor."""
        source_ratio = lambda s: self.source_interp.alpha(s) / self.source_interp.beta(s)
        target_ratio = self.target_interp.alpha(target_t) / (self.target_interp.beta(target_t))
        matched_t = self.binary_search_for_time(function=source_ratio, target=target_ratio)
        scaling_factor = torch.where(
            target_t == 0,
            torch.ones_like(target_t),
            torch.where(
                target_t == 1,
                torch.ones_like(target_t),
                self.source_interp.alpha(matched_t) / self.target_interp.alpha(target_t)
            )
        )
        matched_t = torch.where(
            target_t == 0,
            torch.zeros_like(target_t),
            torch.where(
                target_t == 1,
                torch.ones_like(target_t),
                matched_t
            )
        )
        return matched_t, scaling_factor
    
    def get_transfored_velocity(self, transformed_rf, x_target, t_target):
        """Get the transformed velocity at x_t and time t."""
        t_target = transformed_rf.match_dim_with_data(t_target, x_target.shape, expand_dim=False)
        t_source, scaling_factor = self.match_time_and_scale(t_target)
        scaling_factor = transformed_rf.match_dim_with_data(scaling_factor, x_target.shape, expand_dim=True)
        
        original_velocity = self.rectified_flow.get_velocity(scaling_factor * x_target, t_source)
        result = self.source_interp.solve(t=t_source, x_t=scaling_factor * x_target, dot_x_t=original_velocity)
        
        return self.target_interp.solve(t=t_target, x_0=result.x_0, x_1=result.x_1).dot_x_t
    
    def transform_rectified_flow(self):
        transformed_rf = copy.deepcopy(self.rectified_flow)
        transformed_rf.velocity_field = lambda x, t: self.get_transfored_velocity(transformed_rf, x, t)
        transformed_rf.interp = self.target_interp
        transformed_rf.source_interp = self.target_interp
        return transformed_rf

    @staticmethod
    def binary_search_for_time(function, target, tol=1e-7):
        """ Binary search for the time t such that function(t) = target, tensor version.
        target: torch.Tensor, shape (batch_size,)
        function: takes in a tensor of shape (batch_size,) and returns a tensor of shape (batch_size,)
        """
        lower_bound = torch.zeros_like(target)
        upper_bound = torch.ones_like(target)

        mid_point = (upper_bound + lower_bound) / 2
        current_value = function(mid_point)
        iteration_count = 1
        
        while torch.max(torch.abs(current_value - target)) > tol and torch.max(torch.abs(upper_bound - lower_bound)) > tol:
            upper_bound = torch.where(current_value >= target, mid_point, upper_bound)
            lower_bound = torch.where(current_value <= target, mid_point, lower_bound)
            
            mid_point = (upper_bound + lower_bound) / 2.
            current_value = function(mid_point)
            iteration_count += 1
            
            if iteration_count > 1_000:
                warnings.warn("Binary search exceeded maximum iterations.", UserWarning)
                break
        
        return mid_point

This figure shows that, for a given $ t' $ in $\{X'_{t'}\}$, the corresponding $ t $ in $\{X_t\}$ and the scaling factor $\omega_t = X_t / X'_{t'}$.

In [ ]:
interp_type = "spherical"

convert = AffineInterpConverter(straight_rf, AffineInterp(interp_type))

t = torch.linspace(0, 1, 5000)
matched_t, scaling_factor = convert.match_time_and_scale(t)

fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=matched_t, mode='lines', name='matched_t'))
fig.add_trace(go.Scatter(x=t, y=scaling_factor, mode='lines', name='scaling_factor'))
fig.update_layout(
    title=f'Matched Time and Scaling Factor, {interp_type} Interpolation',
    xaxis_title='t',
    yaxis_title='Value',
    height=500,
    width=700,
)
fig.show()

We have successfully converted the pretrained straight rectified flow into a spherical one.

Next, let's perform sampling with different ODE steps. By varying the `num_steps`, we observe that as the `num_steps` increases, the MSE between the generated $Z_1$ and $Z_1'$ decreases.

In fact, we can actually prove that if $Z_0=Z_0'=X_0$, then $Z_1=Z_1'$, where $Z_1 = \int_0^1 v(Z_t,t) \mathrm d t$ and $Z_1' = \int_0^1 v'(Z_t',t) \mathrm d t$, which means:
$$
(Z_0, Z_1) = (Z_0', Z_1')
$$

In other words, their rectified couplings are equivalent! Despite following very different trajectories, their outcomes turn out to be identical. 

However, even though different interpolations yield the same rectified coupling $(Z_0, Z_1)$, their flows follow different trajectories $\{Z_t\}$. When we simulate these trajectories, we cannot achieve perfect solutions because discretization is required. Generally, straighter trajectories of $\{Z_t\}$ are preferred, as they tend to reduce discretization error.

In [ ]:
target_interp = AffineInterp("spherical")

spherical_rf = AffineInterpConverter(straight_rf, target_interp).transform_rectified_flow()

# Try different num_steps, e.g. [10, 50, 100, 1000]
num_steps = 100
euler_sampler1 = EulerSampler(straight_rf, num_steps=num_steps, num_samples=500)
euler_sampler2 = EulerSampler(spherical_rf, num_steps=num_steps, num_samples=500)

euler_sampler1.sample_loop(seed=0)
euler_sampler2.sample_loop(seed=0)

print("mse", torch.mean((euler_sampler1.trajectories[-1] - euler_sampler2.trajectories[-1])**2))

# zoom in to see they are really close
visualize_2d_trajectories_plotly(
	trajectories_dict={"1rf straight": euler_sampler1.trajectories, "1rf spherical": euler_sampler2.trajectories},
	D1_gt_samples=D1[:5000],
	num_trajectories=100,
	title="Euler Sampler Visualization",
)

# Train Two Rectified Flows: Nearly Same Results!

Another interesting finding (also provable), is that given a sufficiently large dataset, different models or interpolations will converge to the same rectified coupling.

We first demonstrate this phenomenon using a 2D toy example.

In [ ]:
straight_rf = RectifiedFlow(
    data_shape=(2,),
    velocity_field=MLPVelocity(2, hidden_sizes = [128, 128, 128]).to(device),
    interp="straight",
    source_distribution=pi_0,
    device=device,
)

spherical_rf = RectifiedFlow(
    data_shape=(2,),
	velocity_field=MLPVelocity(2, hidden_sizes = [128, 128, 128]).to(device),
	interp="spherical",
	source_distribution=pi_0,
	device=device,
)

In [ ]:
n_samples = 500
pi_0 = dist.MultivariateNormal(torch.zeros(2, device=device), torch.eye(2, device=device))
pi_1 = TwoPointGMM(x=15.0, y=2, std=0.3)
D0 = pi_0.sample([n_samples])
D1, labels = pi_1.sample_with_labels([n_samples])

set_seed(0)
rf_trainer(straight_rf, "straight interp")
set_seed(0)
rf_trainer(spherical_rf, "spherical interp")

sampler1 = EulerSampler(straight_rf, num_steps=1000, num_samples=500)
sampler2 = EulerSampler(spherical_rf, num_steps=1000, num_samples=500)

sampler1.sample_loop(seed=0)
sampler2.sample_loop(seed=0)

print("mse", torch.mean((sampler1.trajectories[-1] - sampler2.trajectories[-1])**2))

visualize_2d_trajectories_plotly(
	trajectories_dict={"straight interp": sampler1.trajectories, "spherical interp": sampler2.trajectories},
	D1_gt_samples=D1[:5000],
	num_trajectories=100,
	title="Euler Sampler Visualization",
)

In [ ]:
def rf_trainer_sample(rectified_flow, label = "loss", batch_size = 1024):
    model = rectified_flow.velocity_field
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    losses = []
    for step in range(6000):
        optimizer.zero_grad()
        x_0 = pi_0.sample([batch_size])
        x_1, labels = pi_1.sample_with_labels([batch_size])
        x_0 = x_0.to(device)
        x_1 = x_1.to(device)

        loss = rectified_flow.get_loss(x_0, x_1)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())

        if step % 200 == 0:
            print(f"Epoch {step}, Loss: {loss.item()}")

    plt.plot(losses, label=label)
    plt.legend()
    
set_seed(0)
rf_trainer_sample(straight_rf, "straight interp")
set_seed(0)
rf_trainer_sample(spherical_rf, "spherical interp")

sampler1 = EulerSampler(straight_rf, num_steps=1000, num_samples=500)
sampler2 = EulerSampler(spherical_rf, num_steps=1000, num_samples=500)

sampler1.sample_loop(seed=0)
sampler2.sample_loop(seed=0)

print("mse", torch.mean((sampler1.trajectories[-1] - sampler2.trajectories[-1])**2))

visualize_2d_trajectories_plotly(
	trajectories_dict={"straight interp": sampler1.trajectories, "spherical interp": sampler2.trajectories},
	D1_gt_samples=D1[:5000],
	num_trajectories=100,
	title="Euler Sampler Visualization",
)